In [ ]:
from pathlib import Path
import numpy as np
from sidecar import Sidecar

from astropy.coordinates import SkyCoord
from mast_aladin_lite import MastAladin
from astroquery.mast.missions import MastMissions
import astropy.units as u
from ipywidgets import widgets, Layout, VBox

Query MAST for data products on M92 from JWST:

In [ ]:
target_name = "TRAPPIST-1"

from astroquery.mast import Catalogs

catalog_data = Catalogs.query_region(
    coordinates=SkyCoord.from_name(target_name),
    radius=1.2*u.arcmin,
    catalog="Gaia",
)
# note: the Hubble Source Catalog doesn't have `ra` and `dec` columns, 
# which are needed by aladin-lite:
# catalog_data.rename_columns(['MatchRA', 'MatchDec'], ['ra', 'dec'])


# find MAST data products
MastMissions.mission = 'jwst'
products = MastMissions.query_criteria(program=1201, exp_type='NRS_WATA', targprop=target_name)

Launch `mast-aladin-lite` and a table viewer with the available data products:

In [ ]:
# create an instance of MastAladin:
mast_aladin = MastAladin(
    target=target_name,
    fov=0.05,
    height=600,
    survey='SDSS9 colored',
)

# open the MastAladin widget on the right:
with Sidecar(title='Aladin', anchor='right'):
    display(
        VBox(
            children=(
                # this displays the aladin-lite widget:
                mast_aladin,
                
                # this displays an "info" widget below, useful
                # for printing and debugging:
                mast_aladin.info
            )
        )
    )

In [ ]:
# at the moment, the regions must be loaded before the catalog. it seems
# like the active layer of "objects" that can be selected is the last-loaded
# layer of "objects"
mast_aladin.add_graphic_overlay_from_stcs(list(products['s_region']))

# now load the table with the method in MastAladin, which
# passes the catalog on through aladin-lite and creates a
# table widget:
mast_aladin.load_table(catalog_data, name='gaia')

# display the table widget below:
with Sidecar(title='Table', anchor='split-bottom'):
    display(mast_aladin.selected_table)

# this demo callback will fire whenever another 
# object in the catalog is selected:
def callback(msg):
    print("this callback will change aladin-lite one day")

# add the callback to a list of callbacks that fire on each change
# in selected objects:
mast_aladin.selected_table.row_select_callbacks = [callback]